In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [3]:
ruta = r"C:\Users\kikot\Desktop\MASTER DATA SCIENCE UAH\9 ESCALADO DE DATOS\PEC 2\titanic.csv"
titanicDF = pd.read_csv(ruta, delimiter=";")

In [4]:
titanicDF.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,survived
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,2113375.00,B5,S,1
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,1
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.55,C22 C26,S,0
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.55,C22 C26,S,0
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.55,C22 C26,S,0


In [5]:
# Obtenemos las columnas con valores NaN y el número de NaN en cada una
columnas_con_nan = titanicDF.columns[titanicDF.isna().any()]
num_nan_por_columna = titanicDF.isna().sum()

# Imprimir los resultados
print("Columnas con NaN:")
for columna in columnas_con_nan:
    num_nan = num_nan_por_columna[columna]
    print(f"{columna}: {num_nan} NaN")

Columnas con NaN:
age: 263 NaN
cabin: 1013 NaN
embarked: 2 NaN


In [6]:
# Eliminamos las columnas que no nos sirven para naday que tienen Nan
titanicDF.drop(["cabin", "embarked"], axis=1, inplace=True) 

In [7]:
# Calculamos la media de las edades sin considerar los valores NaN
media_edades = titanicDF['age'].mean()

# Reemplazamos los valores NaN en la columna "age" por la media de las edades
titanicDF['age'].fillna(media_edades, inplace=True)

In [8]:
# Obtenemos las columnas con valores NaN y el número de NaN en cada una
columnas_con_nan = titanicDF.columns[titanicDF.isna().any()]
num_nan_por_columna = titanicDF.isna().sum()

# Imprimir los resultados
print("Columnas con NaN:")
for columna in columnas_con_nan:
    num_nan = num_nan_por_columna[columna]
    print(f"{columna}: {num_nan} NaN")

Columnas con NaN:


In [9]:
# Vemos que ya tenemos limpio de NAN el DF, y ya hicimos el 1º transform

In [10]:
titanicDF.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,survived
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,2113375.00,1
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,1
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.55,0
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.55,0
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.55,0


In [11]:
# Dividimos el DataFrame en características (X) y variable objetivo (y)
X = titanicDF.drop('survived', axis=1)
y = titanicDF['survived']

# Realizamos la división en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Convertimos la variable "sex" de categórica a numérica (Y asdi hacemos el 2º transform)
label_encoder = LabelEncoder()
X['sex'] = label_encoder.fit_transform(X['sex'])

# Definimos las columnas numéricas y categóricas
columnas_numericas = ['pclass', 'age']
columnas_categoricas = []

# Preprocesamiento de las características
preprocessor = ColumnTransformer(
    transformers=[
        ('numericas', StandardScaler(), columnas_numericas),
        ('categoricas', OneHotEncoder(), columnas_categoricas)
    ])

# Crear el pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression())])

# Ajustar el pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Realizar predicciones utilizando el pipeline ajustado y sacar probabilidades
predictions = pipeline.predict(X_test)
probabilities = pipeline.predict_proba(X_test)[:, 1]

# Crear DataFrame con los resultados
results = pd.DataFrame({'survived': y_test.values, 'prediction': predictions, 'probability': probabilities})

# Mostrar los resultados
print(results.head(10))

   survived  prediction  probability
0         0           0     0.239846
1         1           0     0.424124
2         0           0     0.308888
3         0           0     0.118168
4         0           0     0.233750
5         1           1     0.526169
6         0           0     0.290396
7         0           0     0.168181
8         0           0     0.239846
9         1           1     0.612380


In [13]:
# Aprovecho lo que he trabajado en otra asignatura para sacar precision, exactitud y exhaustiviad del modelo

from sklearn.metrics import precision_score, recall_score, accuracy_score

# Calculamos las predicciones en el conjunto de entrenamiento
y_train_pred = pipeline.predict(X_train)

# Calculamos las predicciones en el conjunto de prueba
y_test_pred = pipeline.predict(X_test)

# Sacamos el precision score en el conjunto de prueba
precision_test = precision_score(y_test, y_test_pred)

# Calculamos el recall score en el conjunto de prueba
recall_test = recall_score(y_test, y_test_pred)

# Calcular el accuracy score en el conjunto de prueba
accuracy_test = accuracy_score(y_test, y_test_pred)

print(f'Precision Score (Prueba): {precision_test}')
print(f'Recall Score (Prueba): {recall_test}')
print(f'Accuracy Score (Prueba): {accuracy_test}')

Precision Score (Prueba): 0.7796610169491526
Recall Score (Prueba): 0.4339622641509434
Accuracy Score (Prueba): 0.7213740458015268


### PREGUNTA 2 Empaquetar el algoritmo, en mi caso uso PKL


In [14]:
from joblib import dump

# Aqui tuve problermas, segun investigue: El objeto Pipeline en scikit-learn no tiene un método directo llamado save() 
# para guardar el modelo en un archivo .pkl, asi que lo hice fue importar dump de joblib y asi ya me deja guardar el algoritmo
# con la extension pkl

dump(pipeline, r"C:\Users\kikot\Desktop\MASTER DATA SCIENCE UAH\9 ESCALADO DE DATOS\PEC 2\pipelineModel.pkl")


['C:\\Users\\kikot\\Desktop\\MASTER DATA SCIENCE UAH\\9 ESCALADO DE DATOS\\PEC 2\\pipelineModel.pkl']

### Disponibilizar el algoritmo como API utilizando Flask dentro de un Contenedor y subirlo a Docker Hub

In [15]:
# Vamos por partes, como lo hizo usted en las clases
# Primero creo un archivo de requirements.txt

flask == 2.1.0
requests
numpy
pandas
joblib
scikit-learn

# Despues como ya tengo creado el pipelineModel.pkl, hago el siguiente Dockerfile

FROM python:3.8
#Set working directory to app
COPY . /app
WORKDIR /app
#Install requirements.txt
RUN pip install -r requirements.txt
EXPOSE 5000
CMD python ./app.py

# Despues ya tengo copiado el template que he adaptado a mi Data Frame:

# index.htlm

</head>
<body>
    <h1>Survival Prediction</h1>
    <form name="survivalForm" action="/" method="POST">
        <label>Sex:</label>
        <select name="sex">
            <option value="male">Male</option>
            <option value="female">Female</option>
        </select>
        <label>Pclass:</label>
        <input type="number" name="pclass" min="1" max="3">
        <label>Age:</label>
        <input type="number" name="age" step="0.1">
        <input type="submit" value="Submit">
    </form>
</body>
</html>

# result.htlm 

<!DOCTYPE html>
<html>
<head>
    <title>Survival Prediction Result</title>
</head>
<body>
    <h1>Survival Prediction Result</h1>
    <p>Prediction: {{ prediction }}</p>
</body>
</html>

# Tambien he creado lo mas importante la app.py con mi df:

from flask import Flask, request, render_template, redirect, url_for
import pandas as pd
import joblib

app = Flask(__name__, template_folder='template')

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        sex = request.form['sex']
        pclass = int(request.form['pclass'])
        age = float(request.form['age'])
        return redirect(url_for('result', sex=sex, pclass=pclass, age=age))
    return render_template('index.html')

@app.route('/result/<string:sex>/<int:pclass>/<float:age>', methods=['GET', 'POST'])
def result(sex, pclass, age):
    # Load the trained pipeline model
    pipeline = joblib.load("pipelineModel.pkl")

    # Create a DataFrame with the input values
    data = pd.DataFrame([[sex, pclass, age]], columns=['sex', 'pclass', 'age'])

    # Make predictions using the pipeline model
    prediction = pipeline.predict(data)[0]

    if prediction == 1:
        prediction = "Survived"
    else:
        prediction = "Not Survived"

    return render_template('result.html', prediction=prediction)

if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=int("5000"))

SyntaxError: invalid syntax (3865905221.py, line 4)